In [7]:
%pip install torch torch-geometric
%pip install pandas

  Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl (7.6 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl (45 kB)

  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

genuine_users = pd.read_csv('./cresci-2017/genuine_accounts/users.csv', encoding_errors='replace')
genuine_tweets = pd.read_csv('cresci-2017/genuine_accounts/tweets.csv', encoding_errors='replace')
social_spam_users_1 = pd.read_csv('cresci-2017/social_spambots_1/users.csv', encoding_errors='replace')
social_spam_tweets_1 = pd.read_csv('cresci-2017/social_spambots_1/tweets.csv', encoding_errors='replace')
social_spam_users_2 = pd.read_csv('cresci-2017/social_spambots_2/users.csv', encoding_errors='replace')
social_spam_tweets_2 = pd.read_csv('cresci-2017/social_spambots_2/tweets.csv', encoding_errors='replace')
social_spam_users_3 = pd.read_csv('cresci-2017/social_spambots_3/users.csv', encoding_errors='replace')
social_spam_tweets_3 = pd.read_csv('cresci-2017/social_spambots_3/tweets.csv', encoding_errors='replace')
traditional_spam_users = pd.read_csv('cresci-2017/traditional_spambots_1/users.csv', encoding_errors='replace')
traditional_spam_tweets = pd.read_csv('cresci-2017/traditional_spambots_1/tweets.csv', encoding_errors='replace')
# fake_follower_users = pd.read_csv('cresci-2017/fake_followers/users.csv', encoding_errors='replace')
# fake_follower_tweets = pd.read_csv('cresci-2017/fake_followers/tweets.csv', encoding_errors='replace')

# # example of getting only reply tweets
# filtered_genuine_tweeets = genuine_tweets[genuine_tweets['in_reply_to_user_id'] != 0.0]

print(genuine_users.columns)
print(genuine_tweets.columns)
print(social_spam_users_1.columns)
print(social_spam_tweets_1.columns)
print(social_spam_users_2.columns)
print(social_spam_tweets_2.columns)
print(social_spam_users_3.columns)
print(social_spam_tweets_3.columns)
print(traditional_spam_users.columns)
print(traditional_spam_tweets.columns)

c:\Users\work\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\work\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\work\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['id', 'name', 'screen_name', 'statuses_count', 'followers_count',
       'friends_count', 'favourites_count', 'listed_count', 'url', 'lang',
       'time_zone', 'location', 'default_profile', 'default_profile_image',
       'geo_enabled', 'profile_image_url', 'profile_banner_url',
       'profile_use_background_image', 'profile_background_image_url_https',
       'profile_text_color', 'profile_image_url_https',
       'profile_sidebar_border_color', 'profile_background_tile',
       'profile_sidebar_fill_color', 'profile_background_image_url',
       'profile_background_color', 'profile_link_color', 'utc_offset',
       'is_translator', 'follow_request_sent', 'protected', 'verified',
       'notifications', 'description', 'contributors_enabled', 'following',
       'created_at', 'timestamp', 'crawled_at', 'updated', 'test_set_1',
       'test_set_2'],
      dtype='object')
Index(['id', 'text', 'source', 'user_id', 'truncated', 'in_reply_to_status_id',
       'in_reply_to_user_id

In [3]:
# Combining data into a single dataframe
genuine_users['label'] = 0
social_spam_users_1['label'] = 1
social_spam_users_2['label'] = 1
social_spam_users_3['label'] = 1
traditional_spam_users['label'] = 1

all_users = pd.concat([
    genuine_users,
    social_spam_users_1,
    social_spam_users_2,
    social_spam_users_3,
    traditional_spam_users
], ignore_index=True)

# all_users.head()

In [4]:
# Preparing relevant features
import numpy as np

features = [
    'statuses_count', 'followers_count', 'friends_count',
    'favourites_count', 'listed_count', 'default_profile',
    'default_profile_image', 'geo_enabled', 'protected', 'verified'
]

for col in features:
    if all_users[col].dtype == bool:
        all_users[col] = all_users[col].astype(int)

X = all_users[features].fillna(0).to_numpy(dtype=np.float32)
y = all_users['label'].to_numpy(dtype=np.int64)


In [6]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import numpy as np

# # Sample graph with 5 nodes
# edge_index = torch.tensor([[0, 1, 2, 3, 4, 1],
#                            [1, 0, 3, 2, 1, 4]], dtype=torch.long)  # directed

# # Node features: tweet frequency, URL presence, hashtag density, etc.
# x = torch.tensor([
#     [0.7, 1.0, 0.5, 0.8],  # user 0
#     [0.2, 0.0, 0.1, 0.4],  # user 1
#     [0.9, 1.0, 0.6, 0.7],  # user 2
#     [0.1, 0.0, 0.2, 0.3],  # user 3
#     [0.8, 1.0, 0.4, 0.9],  # user 4
# ], dtype=torch.float)

# # Labels: 1 = bot, 0 = human
# y = torch.tensor([1, 0, 1, 0, 1], dtype=torch.long)

# # Use a basic graph object
# data = Data(x=x, edge_index=edge_index, y=y)


GNN model


In [7]:
from torch.nn import Linear

class BotClassifier(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.lin = Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.lin(x)
        return x


In [8]:
from sklearn.neighbors import kneighbors_graph

x_tensor = torch.tensor(X)
y_tensor = torch.tensor(y)

adj = kneighbors_graph(X, n_neighbors=5, mode='connectivity', include_self=False)
edge_index = torch.tensor(np.array(adj.nonzero()), dtype=torch.long)
data = Data(x=x_tensor, edge_index=edge_index, y=y_tensor)
model = BotClassifier(input_dim=x_tensor.shape[1], hidden_dim=32, output_dim=2)

Reinfrocement learning

In [ ]:
input_dim = X.shape[1]  # which is 10
model = BotClassifier(input_dim=input_dim, hidden_dim=8, output_dim=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def compute_reward(pred, y_true):
    return (pred.argmax(dim=1) == y_true).float() * 1.0 - 0.5  # +1 or -0.5

for epoch in range(100):
    model.train()
    optimizer.zero_grad()

    out = model(data)
    loss = F.cross_entropy(out, data.y)

    # Simulate "reward" from environment
    reward = compute_reward(out, data.y).mean()

    # RL-inspired update: reward modulates loss
    total_loss = loss - 0.01 * reward  # encourage higher reward
    total_loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        acc = (out.argmax(dim=1) == data.y).float().mean().item()
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}, Acc: {acc:.2f}, Reward: {reward.item():.2f}")



Epoch 0, Loss: 472.8196, Acc: 0.63, Reward: 0.13
Epoch 10, Loss: 23.5165, Acc: 0.43, Reward: -0.07
Epoch 20, Loss: 13.7960, Acc: 0.87, Reward: 0.37
Epoch 30, Loss: 3.4698, Acc: 0.90, Reward: 0.40
Epoch 40, Loss: 1.6609, Acc: 0.92, Reward: 0.42
Epoch 50, Loss: 1.0989, Acc: 0.91, Reward: 0.41
Epoch 60, Loss: 0.7315, Acc: 0.88, Reward: 0.38
Epoch 70, Loss: 0.5465, Acc: 0.88, Reward: 0.38
Epoch 80, Loss: 0.4916, Acc: 0.87, Reward: 0.37
Epoch 90, Loss: 0.4682, Acc: 0.87, Reward: 0.37


In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

model.eval()
preds = model(data).argmax(dim=1)
print("Predicted:", preds.tolist())
print("Actual:   ", data.y.tolist())


y_pred = preds.cpu().numpy()
y_true = data.y.cpu().numpy()

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred)
rec = recall_score(y_true, y_pred)

print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")



Predicted: [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1